In [ ]:
import thermoTools
import thermoToolsAdditions as tta
import json
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from operator import itemgetter
import os

### Example 1: Standalone Thermochimica
This is an example of running a standalone thermochimica calculation for a series of temperatures

In [34]:
dataFile = Path("data/MSTDB-TC_V3.0_Chlorides_No_Functions_8-2.dat")
scriptName = "test"
elements = ['Na', 'Cl', 'Mg', 'Pu']

component_fractions = {'Na Cl': 0.8, 'Mg Cl_2': 0.1, 'Pu Cl_3': 0.1}

tstart = 1500
tend = 2000
ntstep = 20
pstart = 1
pend = 1
npstep = 1
masses = tta.component_fractions_to_element_fractions(component_fractions, elements)

print(masses)

# Write an input script
thermoTools.WriteInputScript(scriptName, str(dataFile), elements, tstart, tend, ntstep, pstart, pend, npstep, masses, heatCapacity=True)

# Run an input script
thermochimicaPath = Path("/home/mlouis9/thermochimica")
outputPath = Path("../../PythonProjects/fuelOptimization/outputs")
outputName = "output.json"

thermoTools.RunInputScript(scriptName, jsonName=str(outputPath / outputName), thermochimica_path=str(thermochimicaPath), noOutput=True)

[0.8, 1.3, 0.1, 0.1]


### Example 2: Solubility Calculation
This is an example solubility calculation for $\text{PuCl}_3$ in the eutectic salt $x(\text{LiCl}):x(\text{KCl}) = (0.586:0.414)$ 

In [39]:
# File IO input parameters
thermochimica_path = Path("/home/mlouis9/thermochimica")
output_path = Path("../../PythonProjects/fuelOptimization/outputs")
output_name = "output.json"
data_file = Path("data/MSTDB-TC_V3.0_Chlorides_No_Functions_8-2.dat")
script_name = "test"

# Physical parameters
temp = 723
press = 1
unit_ratio_of_other_components = {'Li Cl': 0.586, 'K Cl': 0.414}
component_to_vary = 'Pu Cl_3'
compstart = 0.01
compstop = 0.4
n_comp_step = 20
calc = tta.solubility_calculation(temp, press, unit_ratio_of_other_components, component_to_vary, n_comp_step, \
                       thermochimica_path, output_path, output_name, data_file, compstart=compstart, compstop=compstop)

In [36]:
# calc.mole_fraction_element_by_phase['Pu']
list(calc.output.values())[0]['# pure condensed phases']

1

In [37]:
calc.stable_phases

[[('MSCL', 2.9464429866271837), ('K3PuCl6_S1(s)', 0.0005121166296182655)],
 [('MSCL', 2.6956923610962105), ('K3PuCl6_S1(s)', 0.021975139960877986)],
 [('MSCL', 2.4595392096320094), ('K3PuCl6_S1(s)', 0.041578934878305954)],
 [('MSCL', 1.8667239265460243),
  ('FM3M', 0.1114180582706351),
  ('K3PuCl6_S1(s)', 0.06386755993561795)],
 [('FM3M', 0.19541079143371445),
  ('LiCl_FM3M_No.225(s)', 0.5282734190815576),
  ('K2PuCl5_S1(s)', 0.09210526315641229)],
 [('FM3M', 0.5301800809126588),
  ('KCl_FM3M_No.225(s)', 0.1319251822452358),
  ('K2PuCl5_S1(s)', 0.11263157894736847)],
 [('FM3M', 0.509104634745515),
  ('Pu_FCC_A1(s4)', 1.2439677358375583e-15),
  ('K2PuCl5_S1(s)', 0.04173621369287485),
  ('K3PuCl6_S1(s)', 0.09142168104396603)],
 [('FM3M', 0.49704933472166274),
  ('K2PuCl5_S1(s)', 0.1117861768269219),
  ('K3PuCl6_S1(s)', 0.041898033699393)],
 [('FM3M', 0.4839738683160135),
  ('Pu_FCC_A1(s4)', 3.6710377427976033e-16),
  ('PuCl3_P63/M_No.176(s)', 0.0033027236316905285),
  ('K2PuCl5_S1(s)', 0

In [38]:
calc.output['5']['elements']

{'Pu': {'moles': 0.09210526315789473, 'element potential': -900830.5957855599},
 'K': {'moles': 0.3758684210526316, 'element potential': -422505.0752027534},
 'Cl': {'moles': 1.1842105263157894, 'element potential': -85408.22299119676},
 'Li': {'moles': 0.5320263157894737, 'element potential': -376772.7519562255}}